**감성분석 결과 바탕 가공**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
from datetime import datetime, timedelta

In [3]:
eco_senti = pd.read_csv('/content/drive/MyDrive/창종설/senti/감성분석 붙은거.csv')

In [4]:
eco_senti.head()

,Unnamed: 0,게시일,제목,링크,조회수,공감,감정분석결과,감정분석확률,보정된_감정분석확률,감정레이블
0,0,2023-07-04,진대제의 스카이레이크 전문운용사가 에코프로그룹 인정 했습니다,/jongto/253110710?page=55000,177,7,1,0.532199,0.642280,중립
1,1,2023-07-04,지난 주말 알바들이 조급해하는 글을 보고 우리 모두 눈치챘지,/jongto/253110687?page=55000,81,3,1,0.528621,0.581965,중립
2,2,2023-07-04,공견 야근근무 증거금 계산,/jongto/253110556?page=55000,115,9,1,0.524043,0.504769,중립
3,3,2023-07-04,년차 바보 주린이에서 이제는 벗어나려구요,/jongto/253110511?page=55000,121,3,1,0.531490,0.630335,중립
4,4,2023-07-04,잘 매수해놓고 못팔면 도루묵이다,/jongto/253110328?page=55000,25,1,1,0.540013,0.774034,약한 긍정


In [5]:
data = pd.DataFrame()

In [6]:
data['게시일'] = eco_senti['게시일']
data['보정된_감정분석확률'] = eco_senti['보정된_감정분석확률']
data.head()

,게시일,보정된_감정분석확률
0,2023-07-04,0.642280
1,2023-07-04,0.581965
2,2023-07-04,0.504769
3,2023-07-04,0.630335
4,2023-07-04,0.774034


In [7]:
data['게시일'] = pd.to_datetime(data['게시일'])
data_grouped = data.groupby('게시일')['보정된_감정분석확률'].mean().reset_index()
data_grouped.head()

,게시일,보정된_감정분석확률
0,2023-01-20,0.811968
1,2023-01-21,0.642442
2,2023-01-22,0.759137
3,2023-01-23,0.468123
4,2023-01-24,0.561011


In [8]:
data_sorted = data_grouped.sort_values(by='게시일')
data_sorted = data_sorted.reset_index(drop=True)
data_sorted.head()

,게시일,보정된_감정분석확률
0,2023-01-20,0.811968
1,2023-01-21,0.642442
2,2023-01-22,0.759137
3,2023-01-23,0.468123
4,2023-01-24,0.561011


In [9]:
# 토요일, 일요일 값을 월요일로 더하고 평균내기. 해당 날짜의 데이터가 데이터프레임에 있는 경우에만 drop 명령을 수행
data_sorted.set_index('게시일', inplace=True)
for index, row in data_sorted.iterrows():
    if index.weekday() == 5:  # 토요일
        monday_index = index + timedelta(days=2)
        if monday_index in data_sorted.index:
            # data_sorted.at[monday_index, '감정분석확률'] = (data_sorted.at[monday_index, '감정분석확률'] + row['감정분석확률']) / 2
            data_sorted.at[monday_index, '보정된_감정분석확률'] = (data_sorted.at[monday_index, '보정된_감정분석확률'] + row['보정된_감정분석확률']) / 2
            if index in data_sorted.index:  # 인덱스가 존재하는지 확인
                data_sorted.drop(index, inplace=True)
    elif index.weekday() == 6:  # 일요일
        monday_index = index + timedelta(days=1)
        if monday_index in data_sorted.index:
            # data_sorted.at[monday_index, '감정분석확률'] = (data_sorted.at[monday_index, '감정분석확률'] + row['감정분석확률']) / 2
            data_sorted.at[monday_index, '보정된_감정분석확률'] = (data_sorted.at[monday_index, '보정된_감정분석확률'] + row['보정된_감정분석확률']) / 2
            if index in data_sorted.index:  # 인덱스가 존재하는지 확인
                data_sorted.drop(index, inplace=True)


In [10]:
data_sorted.head()

,보정된_감정분석확률
게시일,
2023-01-20,0.811968
2023-01-23,0.657210
2023-01-24,0.561011
2023-01-25,0.536961
2023-01-26,0.565839


In [11]:
data_sorted.to_csv('/content/drive/MyDrive/창종설/senti/eco_sentiscore.csv', index=True, encoding='utf-8')